In [395]:
#!pip install keras==2.12.0


In [396]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import holidays

from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [483]:
from sklearn.model_selection import train_test_split, GridSearchCV, GridSearchCV, TimeSeriesSplit
from sklearn.calibration import calibration_curve

from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam

In [398]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [485]:
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer, mean_squared_error, classification_report


In [400]:
#Set parameters to see all data
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [401]:
# Read dataset

%%time
data_mart_path = 'interview_house_prices.csv'

try:
    data_mart_df = pd.read_csv(data_mart_path)
except pd.errors.ParserError as e:
    print(f'Error while parsing CSV file: {e}')



data_mart_df.shape

CPU times: user 27 ms, sys: 1 ms, total: 28 ms
Wall time: 31.1 ms


(1460, 81)

In [402]:
data_mart_df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
  

In [403]:
# Check for missing values
missing_values = data_mart_df.isnull().sum()
missing_percentage = (missing_values / len(data_mart_df)) * 100

# Print missing values and percentages
missing_df = pd.concat([missing_values, missing_percentage], axis=1, keys=['Missing Values', 'Percentage'])
imputation_features = missing_df[missing_df.Percentage > 0]
print(imputation_features)

# Filter features with missing percentages above 50%
missing_features = missing_df[missing_df['Percentage'] > 50].index.tolist()
print("Features with missing percentage above 50%:", missing_features)

              Missing Values  Percentage
LotFrontage              259   17.739726
Alley                   1369   93.767123
MasVnrType               872   59.726027
MasVnrArea                 8    0.547945
BsmtQual                  37    2.534247
BsmtCond                  37    2.534247
BsmtExposure              38    2.602740
BsmtFinType1              37    2.534247
BsmtFinType2              38    2.602740
Electrical                 1    0.068493
FireplaceQu              690   47.260274
GarageType                81    5.547945
GarageYrBlt               81    5.547945
GarageFinish              81    5.547945
GarageQual                81    5.547945
GarageCond                81    5.547945
PoolQC                  1453   99.520548
Fence                   1179   80.753425
MiscFeature             1406   96.301370
SalePrice                  3    0.205479
Features with missing percentage above 50%: ['Alley', 'MasVnrType', 'PoolQC', 'Fence', 'MiscFeature']


In [404]:
data_mart_df[imputation_features.index].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   LotFrontage   1201 non-null   float64
 1   Alley         91 non-null     object 
 2   MasVnrType    588 non-null    object 
 3   MasVnrArea    1452 non-null   float64
 4   BsmtQual      1423 non-null   object 
 5   BsmtCond      1423 non-null   object 
 6   BsmtExposure  1422 non-null   object 
 7   BsmtFinType1  1423 non-null   object 
 8   BsmtFinType2  1422 non-null   object 
 9   Electrical    1459 non-null   object 
 10  FireplaceQu   770 non-null    object 
 11  GarageType    1379 non-null   object 
 12  GarageYrBlt   1379 non-null   float64
 13  GarageFinish  1379 non-null   object 
 14  GarageQual    1379 non-null   object 
 15  GarageCond    1379 non-null   object 
 16  PoolQC        7 non-null      object 
 17  Fence         281 non-null    object 
 18  MiscFeature   54 non-null   

In [405]:
# Drop mostly missingcolumns from the DataFrame
data_mart_df_V2 = data_mart_df.copy()
data_mart_df_V2.drop(columns=missing_features, inplace=True)
print(data_mart_df_V2.shape)

(1460, 76)


In [407]:
numeric_cols = data_mart_df_V2.select_dtypes(include=['number']).columns
print(numeric_cols)

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice'], dtype='object')


In [408]:
# Methods for missing value imputation
def impute_missing_values(df):
    # Drop rows where SalePrice is missing
    df = df.dropna(subset=['SalePrice'])

    # Fill numeric columns with the average of non-null records
    numeric_cols = df.select_dtypes(include=['number']).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

    # Fill object columns with 'NA'
    object_cols = df.select_dtypes(include=['object']).columns
    df[object_cols] = df[object_cols].fillna('NA')

    return df


In [409]:
data_mart_df_V2 = impute_missing_values(data_mart_df_V2)

<ipython-input-408-5ee4329eb2a4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())
<ipython-input-408-5ee4329eb2a4>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[object_cols] = df[object_cols].fillna('NA')


In [411]:
data_mart_df_V2['SaleCondition'].value_counts()

SaleCondition
Normal     1195
Partial     125
Abnorml     101
Family       20
Alloca       12
AdjLand       4
Name: count, dtype: int64

In [412]:
drop_cols = ['Utilities']
get_dummy_cols = ['Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'BldgType',
                  'RoofStyle', 'Foundation', 'BsmtFinType1', 'BsmtFinType2', 'Functional', 'GarageType',
                  'GarageFinish', 'SaleCondition']
label_enc_cols = ['HouseStyle', 'Neighborhood']
minmax_scale_cols = ['OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
                     'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
                     'GarageCars', 'MoSold', 'LotArea', 'LotFrontage']
year_cols = ['YearBuilt', 'GarageYrBlt', 'YrSold', 'YearRemodAdd']
is_col = ['RoofMatl', 'Heating', 'CentralAir', 'Electrical', 'LowQualFinSF', 'PavedDrive', 'EnclosedPorch',
          '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'SaleType', 'BsmtFinSF2', 'MSZoning']
choice_cols = ['Condition1', 'Condition2', 'Exterior1st', 'Exterior2nd']
map_cols = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'HeatingQC', 'KitchenQual',
            'FireplaceQu', 'GarageQual', 'GarageCond']
bins_cols = ['BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageArea',
             'WoodDeckSF', 'OpenPorchSF', 'MasVnrArea']
map_and_label_cols = ['MSSubClass']

In [413]:
# Combine all columns into a single list
all_cols = map_and_label_cols + drop_cols + get_dummy_cols + label_enc_cols + minmax_scale_cols + year_cols + is_col + choice_cols + map_cols + bins_cols

# Dif of two lists
diff_items = set(data_mart_df_V2.columns) - set(all_cols)

print("Differences :")
print(diff_items)



Differences :
{'SalePrice', 'Id'}


In [414]:
data_mart_df_V2[list(diff_items)].head()

,SalePrice,Id
0,208500.0,1
1,181500.0,2
2,223500.0,3
3,140000.0,4
4,250000.0,5


In [415]:
data_mart_df_V2.shape

(1457, 76)

In [416]:
# Convert categorical variable into dummy/indicator variables
for col in get_dummy_cols:
  data_mart_df_V2 = pd.concat([data_mart_df_V2, pd.get_dummies(data_mart_df_V2[col],
                                                               prefix=f"{col}_cat")], axis=1)



# Drop the original categorical columns
data_mart_df_V2.drop(get_dummy_cols, axis=1, inplace=True)
data_mart_df_V2.shape

(1457, 135)

In [417]:
# LabelEncoder
label_encoder = LabelEncoder()

for col in label_enc_cols:
  # Fit LabelEncoder
  data_mart_df_V2[col] = label_encoder.fit_transform(data_mart_df_V2[col])

# Drop the original columns
data_mart_df_V2.drop(columns=label_enc_cols, inplace=True)

In [418]:
# Initialize MinMaxScaler
scaler = MinMaxScaler()

In [419]:
# Method for normalization to numeric columns
def normalize_data(dataframe, columns):

    normalized_dataframe = dataframe.copy()

    # Check if the specified columns exist in the DataFrame
    existing_columns = [col for col in columns if col in normalized_dataframe.columns]

    if existing_columns:
        # Fit and transform the specified columns using MinMaxScaler
        normalized_dataframe[existing_columns] = scaler.fit_transform(normalized_dataframe[existing_columns])

    return normalized_dataframe

In [420]:
data_mart_df_V2[minmax_scale_cols].head()

,OverallQual,OverallCond,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,MoSold,LotArea,LotFrontage
0,7,5,1,0,2,1,3,1,8,0,2,2,8450,65.0
1,6,8,0,1,2,0,3,1,6,1,2,5,9600,80.0
2,7,5,1,0,2,1,3,1,6,1,2,9,11250,68.0
3,7,5,1,0,1,0,3,1,7,1,3,2,9550,60.0
4,8,5,1,0,2,1,4,1,9,1,3,12,14260,84.0


In [421]:
data_mart_df_V2 = normalize_data(data_mart_df_V2, minmax_scale_cols)
data_mart_df_V2[minmax_scale_cols].head()

,OverallQual,OverallCond,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,MoSold,LotArea,LotFrontage
0,0.666667,0.500,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.500000,0.000000,0.50,0.090909,0.033420,0.150685
1,0.555556,0.875,0.000000,0.5,0.666667,0.0,0.375,0.333333,0.333333,0.333333,0.50,0.363636,0.038795,0.202055
2,0.666667,0.500,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.333333,0.333333,0.50,0.727273,0.046507,0.160959
3,0.666667,0.500,0.333333,0.0,0.333333,0.0,0.375,0.333333,0.416667,0.333333,0.75,0.090909,0.038561,0.133562
4,0.777778,0.500,0.333333,0.0,0.666667,0.5,0.500,0.333333,0.583333,0.333333,0.75,1.000000,0.060576,0.215753


In [423]:
# Control for train and test split acording to  Year Built Year
data_mart_df_V2[data_mart_df_V2.YearBuilt < 2000].SalePrice.mean(), data_mart_df_V2[data_mart_df_V2.YearBuilt > 2000].SalePrice.mean()

(175652.85140186915, 306498.0578512397)

In [424]:
#Calculate the year difference between each column value and the minimum value of that column
#in a pandas DataFrame and drop the original columns.

def calculate_year_difference_and_drop(df, year_cols):
    for col in year_cols:
        df[col + '_year_diff'] = df[col].apply(lambda x: x - df[col].min())
    df.drop(columns=year_cols, inplace=True)
    return df

In [425]:
# Get year difference
data_mart_df_V2 = calculate_year_difference_and_drop(data_mart_df_V2, year_cols)
data_mart_df_V2[['YearBuilt_year_diff',	'GarageYrBlt_year_diff',
                 'YrSold_year_diff',	'YearRemodAdd_year_diff']].head()

,YearBuilt_year_diff,GarageYrBlt_year_diff,YrSold_year_diff,YearRemodAdd_year_diff
0,131,103.0,2,53
1,104,76.0,1,26
2,129,101.0,2,52
3,43,98.0,0,20
4,128,100.0,2,50


In [426]:
data_mart_df_V2 = normalize_data(data_mart_df_V2, ['YearBuilt_year_diff',	'GarageYrBlt_year_diff',
                 'YrSold_year_diff',	'YearRemodAdd_year_diff'])

data_mart_df_V2[['YearBuilt_year_diff',	'GarageYrBlt_year_diff',
                 'YrSold_year_diff',	'YearRemodAdd_year_diff']].head()

,YearBuilt_year_diff,GarageYrBlt_year_diff,YrSold_year_diff,YearRemodAdd_year_diff
0,0.949275,0.936364,0.50,0.883333
1,0.753623,0.690909,0.25,0.433333
2,0.934783,0.918182,0.50,0.866667
3,0.311594,0.890909,0.00,0.333333
4,0.927536,0.909091,0.50,0.833333


In [428]:
# Convert boolean col from target
data_mart_df_V2['is_CompShg'] = data_mart_df_V2['RoofMatl'].apply(lambda x: 1 if x == 'CompShg' else 0)
data_mart_df_V2['is_GasA'] = data_mart_df_V2['Heating'].apply(lambda x: 1 if x == 'GasA' else 0)
data_mart_df_V2['is_CentralAir_Y'] = data_mart_df_V2['CentralAir'].apply(lambda x: 1 if x == 'Y' else 0)
data_mart_df_V2['is_SBrkr'] = data_mart_df_V2['Electrical'].apply(lambda x: 1 if x == 'SBrkr' else 0)
data_mart_df_V2['is_LowQualFinSF_0'] = data_mart_df_V2['LowQualFinSF'].apply(lambda x: 0 if x == 0 else 1)
data_mart_df_V2['is_PavedDrive'] = data_mart_df_V2['PavedDrive'].apply(lambda x: 1 if x == 'Y' else 0)
data_mart_df_V2['is_EnclosedPorch_0'] = data_mart_df_V2['EnclosedPorch'].apply(lambda x: 0 if x == 0 else 1)
data_mart_df_V2['is_3SsnPorch_0'] = data_mart_df_V2['3SsnPorch'].apply(lambda x: 0 if x == 0 else 1)
data_mart_df_V2['is_ScreenPorch_0'] = data_mart_df_V2['ScreenPorch'].apply(lambda x: 0 if x == 0 else 1)
data_mart_df_V2['is_PoolArea_0'] = data_mart_df_V2['PoolArea'].apply(lambda x: 0 if x == 0 else 1)
data_mart_df_V2['is_MiscVal_0'] = data_mart_df_V2['MiscVal'].apply(lambda x: 0 if x == 0 else 1)
data_mart_df_V2['is_WD'] = data_mart_df_V2['SaleType'].apply(lambda x: 1 if x == 'WD' else 0)
data_mart_df_V2['is_BsmtFinSF2_0'] = data_mart_df_V2['BsmtFinSF2'].apply(lambda x: 0 if x == 0 else 1)
data_mart_df_V2['is_RL'] = data_mart_df_V2['MSZoning'].apply(lambda x: 1 if x == 'RL' else 0)

# Drop original columns
data_mart_df_V2.drop(columns=is_col, inplace=True)

In [429]:
data_mart_df_V2[['is_GasA', 'is_LowQualFinSF_0', 'is_PoolArea_0', 'is_WD']].head()

,is_GasA,is_LowQualFinSF_0,is_PoolArea_0,is_WD
0,1,0,0,1
1,1,0,0,1
2,1,0,0,1
3,1,0,0,1
4,1,0,0,1


In [432]:
# Transform first and second choice
def transform_data(df, col1, col2, weight_C1 = 0.6, weight_C2 = 0.4):

    # Get unique values of First and Second choices
    unique_values = set(df[col1].unique()) | set(df[col2].unique())

    # Add column for each unique value with weighted value
    for val in unique_values:
        df[f'Choice_{val}'] = ((df[col1] == val) * weight_C1) + ((df[col2] == val) * weight_C2)
    return df

In [433]:
# Transform dataframe
data_mart_df_V3 = data_mart_df_V2.copy()
data_mart_df_V3 = transform_data(data_mart_df_V3, 'Condition1', 'Condition2')
data_mart_df_V3 = transform_data(data_mart_df_V3, 'Exterior1st',	'Exterior2nd')


In [434]:
# Select features which starts with 'Choice_'
choice_cols = [col for col in data_mart_df_V3.columns if col.startswith('Choice_')]

# Display selected features
data_mart_df_V3[choice_cols].head(10)


,Choice_RRNn,Choice_Artery,Choice_PosN,Choice_RRAn,Choice_PosA,Choice_Feedr,Choice_Norm,Choice_RRNe,Choice_RRAe,Choice_Wd Sdng,Choice_Stone,Choice_Wd Shng,Choice_Brk Cmn,Choice_Stucco,Choice_Other,Choice_CemntBd,Choice_AsbShng,Choice_BrkFace,Choice_ImStucc,Choice_CBlock,Choice_VinylSd,Choice_CmentBd,Choice_WdShing,Choice_Plywood,Choice_AsphShn,Choice_HdBoard,Choice_BrkComm,Choice_MetalSd
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.6,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.6,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.6,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,0.0,0.6,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [436]:
# Get unique values
unique_sales_values = set(data_mart_df_V3['ExterQual'].unique()) | set(data_mart_df_V3['ExterCond'].unique()) | set(data_mart_df_V3['BsmtQual'].unique()) | set(data_mart_df_V3['BsmtCond'].unique()) | set(data_mart_df_V3['BsmtExposure'].unique()) | set(data_mart_df_V3['HeatingQC'].unique()) | set(data_mart_df_V3['KitchenQual'].unique()) | set(data_mart_df_V3['FireplaceQu'].unique()) | set(data_mart_df_V3['GarageQual'].unique()) | set(data_mart_df_V3['GarageCond'].unique())
unique_sales_values

{'Av', 'Ex', 'Fa', 'Gd', 'Mn', 'NA', 'No', 'Po', 'TA'}

In [437]:
# Define a dictionary mapping satisfaction values to a standardized representation
satisfaction_values = {
    'Ex': 0.9,
    'Gd': 0.7,
    'TA': 0.5,
    'Fa': 0.3,
    'Av': 0.6,
    'Mn': 0.1,
    'NA': 0,
    'No': 0,
    'Po': 0.2
}

# Apply the mapping to merge similar values
data_mart_df_V4 = data_mart_df_V3.copy()
for col in map_cols:
  data_mart_df_V4[col] = data_mart_df_V4[col].map(satisfaction_values)

# Check unique values after corrections and merging
print(data_mart_df_V4[map_cols])


      ExterQual  ExterCond  BsmtQual  BsmtCond  BsmtExposure  HeatingQC  KitchenQual  FireplaceQu  GarageQual  GarageCond
0           0.7        0.5       0.7       0.5           0.0        0.9          0.7          0.0         0.5         0.5
1           0.5        0.5       0.7       0.5           0.7        0.9          0.5          0.5         0.5         0.5
2           0.7        0.5       0.7       0.5           0.1        0.9          0.7          0.5         0.5         0.5
3           0.5        0.5       0.5       0.7           0.0        0.7          0.7          0.7         0.5         0.5
4           0.7        0.5       0.7       0.5           0.6        0.9          0.7          0.5         0.5         0.5
...         ...        ...       ...       ...           ...        ...          ...          ...         ...         ...
1455        0.5        0.5       0.7       0.5           0.0        0.9          0.5          0.5         0.5         0.5
1456        0.5        0

In [439]:
# Create bins for MasVnrArea
bins = [data_mart_df_V4['MasVnrArea'].min(),
        data_mart_df_V4['MasVnrArea'].astype(int).mean(),
        data_mart_df_V4['MasVnrArea'].astype(int).mean() + data_mart_df_V4['MasVnrArea'].astype(int).std(),
        data_mart_df_V4['MasVnrArea'].max()]

# Split dataset acording to the bins
bins_df = pd.cut(data_mart_df_V4['MasVnrArea'], bins=bins)

# Create groups with bins
data_shape = data_mart_df_V4.groupby(bins_df).size().reset_index(name='count')

print("Bins and sizes:")
print(data_shape)

Bins and sizes:
           MasVnrArea  count
0      (0.0, 103.884]    121
1  (103.884, 284.583]    281
2   (284.583, 1600.0]    196


In [440]:
# Replace numeric columns in the dataframe with bin codes based on specified bin ranges.

def replace_columns_with_bins(dataframe, column_list):

    for column_name in column_list:
        print(column_name)
        # Check if the column exists and is numeric
        if column_name in dataframe.columns and pd.api.types.is_numeric_dtype(dataframe[column_name]):
            # Define bin ranges based on min, mean, mean+std, and max values
            bin_ranges = [dataframe[column_name].min(),
                          dataframe[column_name].astype(int).mean(),
                          dataframe[column_name].astype(int).mean() + dataframe[column_name].astype(int).std(),
                          dataframe[column_name].max()]
            # Replace the numeric values with bin codes
            dataframe[column_name] = pd.cut(dataframe[column_name],
                                            bins=bin_ranges,
                                            include_lowest=True,
                                            duplicates='drop').cat.codes
    return dataframe


In [441]:
data_mart_df_V5 = replace_columns_with_bins(data_mart_df_V4, bins_cols)
data_mart_df_V5[bins_cols].head()

BsmtFinSF1
BsmtUnfSF
TotalBsmtSF
1stFlrSF
2ndFlrSF
GrLivArea
GarageArea
WoodDeckSF
OpenPorchSF
MasVnrArea


,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,MasVnrArea
0,1,0,0,0,2,1,1,0,1,1
1,2,0,1,1,0,0,0,2,0,0
2,1,0,0,0,2,1,1,0,0,1
3,0,0,0,0,1,1,1,0,0,0
4,1,0,1,0,2,2,2,1,1,2


In [443]:

sub_class_dict = {20:	'1-STORY 1946 & NEWER ALL STYLES',
                  30:	'1-STORY 1945 & OLDER',
                  40:	'1-STORY W/FINISHED ATTIC ALL AGES',
                  45:	'1-1/2 STORY - UNFINISHED ALL AGES',
                  50:	'1-1/2 STORY FINISHED ALL AGES',
                  60:	'2-STORY 1946 & NEWER',
                  70:	'2-STORY 1945 & OLDER',
                  75:	'2-1/2 STORY ALL AGES',
                  80:	'SPLIT OR MULTI-LEVEL',
                  85:	'SPLIT FOYER',
                  90:	'DUPLEX - ALL STYLES AND AGES',
                  120:	'1-STORY PUD (Planned Unit Development) - 1946 & NEWER',
                  150:	'1-1/2 STORY PUD - ALL AGES',
                  160:	'2-STORY PUD - 1946 & NEWER',
                  180:	'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER',
                  190:	'2 FAMILY CONVERSION - ALL STYLES AND AGES'
                  }
# Apply the mapping to merge similar values
data_mart_df_V6 = data_mart_df_V5.copy()
for col in map_and_label_cols:
  data_mart_df_V6[col] = data_mart_df_V6[col].map(sub_class_dict)
  # Fit LabelEncoder
  data_mart_df_V6[col] = label_encoder.fit_transform(data_mart_df_V6[col])
  # Fit and transform the specified columns using MinMaxScaler
  data_mart_df_V6[col] = scaler.fit_transform(data_mart_df_V6[col].values.reshape(-1, 1))


data_mart_df_V6[map_and_label_cols].head()


,MSSubClass
0,0.642857
1,0.214286
2,0.642857
3,0.571429
4,0.642857


In [444]:
# Convert values whick has '_cat_ with 0 and 1
for col in data_mart_df_V6.columns:
    if '_cat_' in col:
        data_mart_df_V6[col] = data_mart_df_V6[col].astype(int)

In [445]:
drop_columns = ['Utilities', 'Condition1', 'Condition2', 'Exterior1st', 'Exterior2nd']
data_mart_df_V6.drop(columns=drop_columns, inplace=True)


In [446]:
data_mart_df_V6.iloc[:, :50].head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,MoSold,SalePrice,Street_cat_Grvl,Street_cat_Pave,LotShape_cat_IR1,LotShape_cat_IR2,LotShape_cat_IR3,LotShape_cat_Reg,LandContour_cat_Bnk,LandContour_cat_HLS,LandContour_cat_Low,LandContour_cat_Lvl,LotConfig_cat_Corner,LotConfig_cat_CulDSac,LotConfig_cat_FR2
0,1,0.642857,0.150685,0.033420,0.666667,0.500,1,0.7,0.5,0.7,0.5,0.0,1,0,0,0.9,0,2,1,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.7,0.500000,0.000000,0.0,0.50,1,0.5,0.5,0,1,0.090909,208500.0,0,1,0,0,0,1,0,0,0,1,0,0,0
1,2,0.214286,0.202055,0.038795,0.555556,0.875,0,0.5,0.5,0.7,0.5,0.7,2,0,1,0.9,1,0,0,0.000000,0.5,0.666667,0.0,0.375,0.333333,0.5,0.333333,0.333333,0.5,0.50,0,0.5,0.5,2,0,0.363636,181500.0,0,1,0,0,0,1,0,0,0,1,0,0,1
2,3,0.642857,0.160959,0.046507,0.666667,0.500,1,0.7,0.5,0.7,0.5,0.1,1,0,0,0.9,0,2,1,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.7,0.333333,0.333333,0.5,0.50,1,0.5,0.5,0,0,0.727273,223500.0,0,1,1,0,0,0,0,0,0,1,0,0,0
3,4,0.571429,0.133562,0.038561,0.666667,0.500,0,0.5,0.5,0.5,0.7,0.0,0,0,0,0.7,0,1,1,0.333333,0.0,0.333333,0.0,0.375,0.333333,0.7,0.416667,0.333333,0.7,0.75,1,0.5,0.5,0,0,0.090909,140000.0,0,1,1,0,0,0,0,0,0,1,1,0,0
4,5,0.642857,0.215753,0.060576,0.777778,0.500,2,0.7,0.5,0.7,0.5,0.6,1,0,1,0.9,0,2,2,0.333333,0.0,0.666667,0.5,0.500,0.333333,0.7,0.583333,0.333333,0.5,0.75,2,0.5,0.5,1,1,1.000000,250000.0,0,1,1,0,0,0,0,0,0,1,0,0,1


In [448]:
train, test = train_test_split(data_mart_df_V6, test_size=0.2, random_state=42)

In [449]:
train.shape, test.shape

((1165, 156), (292, 156))

In [452]:
# Create result dataframe
pred_df = pd.DataFrame()
pred_df['Id'] = test['Id'].copy()

In [453]:
# Defining a method to move target data to the end of the dataframe
def move_target_to_end(df, target_column):

    # Ensure the target column is in the DataFrame
    if target_column not in df.columns:
        print(f"Error: Target column '{target_column}' not found in the DataFrame.")
        return df

    # Reorder columns to move the target column to the end
    new_order = [col for col in df.columns if col != target_column] + [target_column]
    df = df[new_order]

    return df

In [454]:
# Move target column to the end of list
train_df = move_target_to_end(train, 'SalePrice')
test_df = move_target_to_end(test, 'SalePrice')


In [455]:
%%time
# Calculating correlation
corr = train_df.corr()
cr = corr.copy()
top_corr_columns = []
#Determine best correlate columns over 0.1
top_corr_columns = cr.loc[:, 'SalePrice'][:-1]
best_accurate_columns = top_corr_columns[abs(top_corr_columns) > 0.01].sort_values(ascending=False)


len(best_accurate_columns)

CPU times: user 125 ms, sys: 0 ns, total: 125 ms
Wall time: 145 ms


94

In [456]:
best_accurate_columns.keys()

Index(['OverallQual', 'GrLivArea', 'FullBath', 'BsmtQual', 'ExterQual', 'TotRmsAbvGrd', 'GarageCars', '2ndFlrSF', 'KitchenQual', 'BsmtFinType1_cat_GLQ', 'is_ScreenPorch_0', 'GarageFinish_cat_Fin', 'Foundation_cat_PConc', 'HeatingQC', 'GarageArea', 'GarageYrBlt_year_diff', 'TotalBsmtSF', 'FireplaceQu', 'Choice_VinylSd', '1stFlrSF', 'BedroomAbvGr', 'LotArea', 'YearBuilt_year_diff', 'HalfBath', 'GarageFinish_cat_RFn', 'YearRemodAdd_year_diff', 'GarageQual', 'GarageCond', 'is_SBrkr', 'is_CentralAir_Y', 'OpenPorchSF', 'BsmtUnfSF', 'LotFrontage', 'BldgType_cat_1Fam', 'Fireplaces', 'Foundation_cat_BrkTil', 'BsmtFinType2_cat_Unf', 'Choice_Norm', 'GarageType_cat_Attchd', 'BsmtCond', 'Functional_cat_Typ', 'Choice_Wd Sdng', 'SaleCondition_cat_Partial', 'LotConfig_cat_Inside', 'BsmtExposure', 'is_GasA', 'MasVnrArea', 'GarageType_cat_BuiltIn', 'MSSubClass', 'Functional_cat_Maj2', 'Choice_WdShing', 'LotShape_cat_IR1', 'BsmtHalfBath', 'BsmtFinType2_cat_LwQ', 'Choice_Wd Shng', 'Functional_cat_Min1',
 

In [457]:
# Subselect dataframe with most correlated fetaures
train_df2 = train_df[best_accurate_columns.keys()].copy()
test_df2 = test_df[best_accurate_columns.keys()].copy()

In [459]:
X_train = train_df2.copy() #features (independent variables)
X_test = test_df2.copy() #features (independent variables)

y_train = train_df['SalePrice'] #target (dependent variable)
y_test = test_df['SalePrice'] #target (dependent variable)

In [460]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1165, 94), (292, 94), (1165,), (292,))

In [461]:

pred_df['Actual_SalePrice'] = y_test
pred_df.head()

,Id,Actual_SalePrice
499,500,120000.0
1264,1265,181000.0
413,414,115000.0
1049,1050,84900.0
1036,1037,315500.0


## XGB Regressor

In [462]:
# Define parameters
 {'gamma': 0.3,
 'learning_rate': 0.001,
 'max_depth': 12,
 'n_estimators': 10,
 'nthread': -1,
 'reg_alpha': 1,
 'reg_lambda': 1,
 'seed': 10}

{'gamma': 0.3,
 'learning_rate': 0.001,
 'max_depth': 12,
 'n_estimators': 10,
 'nthread': -1,
 'reg_alpha': 1,
 'reg_lambda': 1,
 'seed': 10}

In [463]:
# Define XGBRegressor

%%time
xgb = XGBRegressor(enable_categorical=True)

# Define parameter grid for grid search
param_grid = { 'gamma' : [0.3, 0.1, 0.5], 'learning_rate' : [0.0005, 0.001, 0.01],
              'max_depth' : [10, 12, 15],
              'n_estimators' : [5, 10, 25],
              'nthread' : [-1], 'reg_alpha' : [1], 'reg_lambda' : [1], 'seed' : [10]
              }

# Perform grid search with time series cross-validation
#tscv = TimeSeriesSplit(n_splits=-1)
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, verbose=True)
grid_search.fit(X_train, y_train, eval_set=[(X_train, y_train)], early_stopping_rounds=50)


Fitting 5 folds for each of 81 candidates, totalling 405 fits
[0]	validation_0-rmse:852661.82718
[1]	validation_0-rmse:852447.88236
[2]	validation_0-rmse:852234.11314
[3]	validation_0-rmse:852020.45244
[4]	validation_0-rmse:851806.86626
[0]	validation_0-rmse:852753.88565
[1]	validation_0-rmse:852623.93705
[2]	validation_0-rmse:852493.98869
[3]	validation_0-rmse:852364.09158
[4]	validation_0-rmse:852234.27913


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-rmse:852839.11472
[1]	validation_0-rmse:852751.51939
[2]	validation_0-rmse:852663.90126
[3]	validation_0-rmse:852576.37845
[4]	validation_0-rmse:852488.88372
[0]	validation_0-rmse:852629.87820
[1]	validation_0-rmse:852415.39951
[2]	validation_0-rmse:852201.04347
[3]	validation_0-rmse:851986.72596


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[4]	validation_0-rmse:851772.54836
[0]	validation_0-rmse:852618.16542
[1]	validation_0-rmse:852403.32045
[2]	validation_0-rmse:852188.56010
[3]	validation_0-rmse:851973.85103
[4]	validation_0-rmse:851759.17199
[0]	validation_0-rmse:852661.82718
[1]	validation_0-rmse:852447.88236
[2]	validation_0-rmse:852234.11314
[3]	validation_0-rmse:852020.45244
[4]	validation_0-rmse:851806.86626


...

[10]	validation_0-rmse:807351.93910
[11]	validation_0-rmse:803394.72123
[12]	validation_0-rmse:799466.90812
[13]	validation_0-rmse:795567.79542
[14]	validation_0-rmse:791698.07868
[15]	validation_0-rmse:787856.65350
[16]	validation_0-rmse:784044.36108
[17]	validation_0-rmse:780260.82151
[18]	validation_0-rmse:776506.49602
[19]	validation_0-rmse:772779.71312
[20]	validation_0-rmse:769082.00189
[21]	validation_0-rmse:765411.95993
[22]	validation_0-rmse:761770.44110
[23]	validation_0-rmse:758157.18432
[24]	validation_0-rmse:754571.31897
[0]	validation_0-rmse:848535.09226
[1]	validation_0-rmse:844259.19573
[2]	validation_0-rmse:840005.22609
[3]	validation_0-rmse:835773.15078
[4]	validation_0-rmse:831563.48506
[5]	validation_0-rmse:827374.66088
[6]	validation_0-rmse:823208.09689
[7]	validation_0-rmse:819062.20969
[8]	validation_0-rmse:814938.19671


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[9]	validation_0-rmse:810834.50775
[10]	validation_0-rmse:806751.11093
[11]	validation_0-rmse:802689.83430
[12]	validation_0-rmse:798648.89995
[13]	validation_0-rmse:794628.90981
[14]	validation_0-rmse:790628.18054
[15]	validation_0-rmse:786649.62149
[16]	validation_0-rmse:782690.38619
[17]	validation_0-rmse:778767.67533
[18]	validation_0-rmse:774876.86101
[19]	validation_0-rmse:771017.93484
[20]	validation_0-rmse:767190.64693
[21]	validation_0-rmse:763394.79017
[22]	validation_0-rmse:759630.54252
[23]	validation_0-rmse:755898.67671
[24]	validation_0-rmse:752198.09885
[0]	validation_0-rmse:852375.80367
[1]	validation_0-rmse:851946.28966
[2]	validation_0-rmse:851516.96651
[3]	validation_0-rmse:851087.92823
[4]	validation_0-rmse:850659.07900
CPU times: user 2min 3s, sys: 2.26 s, total: 2min 5s
Wall time: 2min 18s


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=True, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, ma...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'gamma': [0.3, 0.1, 0.5],
                         'learning_rate': [0.0005, 0.001, 0.01],
                         'max_depth': [10, 12, 15], 'n_estimators': [5, 10, 25],
                         'nthread': [-1], 'reg_alpha': [1], 'reg_lambda': [1],
                         'seed': [10]},
             verbose=True)

In [464]:
grid_search.best_params_

{'gamma': 0.3,
 'learning_rate': 0.001,
 'max_depth': 10,
 'n_estimators': 5,
 'nthread': -1,
 'reg_alpha': 1,
 'reg_lambda': 1,
 'seed': 10}

In [465]:
grid_search.best_score_

-0.18180595305602817

In [466]:
# Error metric definition
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate absolute percentage error
    absolute_percentage_error = np.abs((y_true - y_pred) / y_true)

    # Handle division by zero (replace with zero)
    absolute_percentage_error[np.isinf(absolute_percentage_error)] = 0

    # Calculate mean absolute percentage error
    mape = np.mean(absolute_percentage_error) * 100

    return mape


In [467]:
%%time
# Predict with X_test data
y_pred = grid_search.predict(X_test)

# Calculate RMSE and MAPE
rmse = np.sqrt(mean_squared_error(y_test.astype(np.int32), y_pred.astype(np.int32)))
mape = mean_absolute_percentage_error(y_test.astype(np.int32), y_pred.astype(np.int32))

print("RMSE:", rmse)
print("MAPE:", mape)


RMSE: 12143.627295434053
MAPE: 57.4939620309721
CPU times: user 24.4 ms, sys: 17 µs, total: 24.4 ms
Wall time: 29.5 ms


In [468]:
pred_df['XGB_Preds'] = np.round(y_pred).astype(int)
pred_df.to_csv("preds.csv", index=False)

pred_df.head(10)

,Id,Actual_SalePrice,XGB_Preds
499,500,120000.0,218008
1264,1265,181000.0,218114
413,414,115000.0,217974
1049,1050,84900.0,218062
1036,1037,315500.0,219246
615,616,137500.0,218234
219,220,167240.0,218462
1398,1399,138000.0,218033
1194,1195,158000.0,218008
887,888,135500.0,218008


In [469]:
pred_df.head(10)

,Id,Actual_SalePrice,XGB_Preds
499,500,120000.0,218008
1264,1265,181000.0,218114
413,414,115000.0,217974
1049,1050,84900.0,218062
1036,1037,315500.0,219246
615,616,137500.0,218234
219,220,167240.0,218462
1398,1399,138000.0,218033
1194,1195,158000.0,218008
887,888,135500.0,218008


##Recurrent Neural Network

In [470]:
# Define training parameters
lr = 0.5
dropout_rate = 0.2
num_epochs = 750 #MAPE: 97

# Prepare the data
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

# Convert y_train Series to a NumPy array and reshape it
y_train_array = np.array(y_train).reshape(-1, 1)
# Convert the NumPy array to a PyTorch tensor
y_train_tensor = torch.tensor(y_train_array, dtype=torch.float32)

# Convert y_train Series to a NumPy array and reshape it
y_test_array = np.array(y_test).reshape(-1, 1)
# Convert the NumPy array to a PyTorch tensor
y_test_tensor = torch.tensor(y_test_array, dtype=torch.float32)

# Define the RNN Model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size)  # Reset the initial hidden state to zero
        out, _ = self.rnn(x, h0)  # Apply the RNN layer
        #out = self.Dropout(out)
        #out = self.elu(out)
        out = self.fc(out[:, -1, :])  # Apply the fully connected layer by taking the output of the last time step
        return out

# Model parameters
input_size = X_train.shape[1]  # Input size
hidden_size = 512  # Hidden layer size
output_size = 1  # Output size (prediction)

# Create the model
model = RNN(input_size, hidden_size, output_size)


In [471]:
%%time
# Reshaping the training and test data
X_train_tensor = X_train_tensor.view(-1, 1, input_size)
X_test_tensor = X_test_tensor.view(-1, 1, input_size)

# Define optimization and loss function
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.5) #lr=0.1->MAPE 74, lr=0.5->MAPE 30

# Training loop
#num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs.squeeze(), y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# Test
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    test_loss = criterion(outputs.squeeze(), y_test_tensor)
print("Test Loss:", test_loss.item())


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1165, 1])) that is different to the input size (torch.Size([1165])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/750, Loss: 775008485376.0000
Epoch 2/750, Loss: 775002980352.0000
Epoch 3/750, Loss: 774942228480.0000
Epoch 4/750, Loss: 774862143488.0000
Epoch 5/750, Loss: 774772817920.0000
Epoch 6/750, Loss: 774677659648.0000
Epoch 7/750, Loss: 774579486720.0000
Epoch 8/750, Loss: 774478430208.0000
Epoch 9/750, Loss: 774375997440.0000
Epoch 10/750, Loss: 774271926272.0000
Epoch 11/750, Loss: 774167068672.0000
Epoch 12/750, Loss: 774061096960.0000
Epoch 13/750, Loss: 773954732032.0000
Epoch 14/750, Loss: 773848104960.0000
Epoch 15/750, Loss: 773740953600.0000
Epoch 16/750, Loss: 773633540096.0000
Epoch 17/750, Loss: 773525667840.0000
Epoch 18/750, Loss: 773417861120.0000
Epoch 19/750, Loss: 773310119936.0000
Epoch 20/750, Loss: 773201985536.0000
Epoch 21/750, Loss: 773093720064.0000
Epoch 22/750, Loss: 772985782272.0000
Epoch 23/750, Loss: 772877713408.0000
Epoch 24/750, Loss: 772769579008.0000
Epoch 25/750, Loss: 772661837824.0000
Epoch 26/750, Loss: 772553834496.0000
Epoch 27/750, Loss: 7

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([292, 1])) that is different to the input size (torch.Size([292])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [472]:
# Making predictions on the test data
with torch.no_grad():
    outputs = model(X_test_tensor)
    predictions = outputs.squeeze().numpy()
y_test_array = y_test.values

# Comparing predictions with the actual values
for i in range(30):  # Inspecting the first 30 examples
    print("Actual Value:", y_test_array[i], "\tPrediction:", predictions[i])


Actual Value: 120000.0 	Prediction: 148459.03
Actual Value: 181000.0 	Prediction: 148459.03
Actual Value: 115000.0 	Prediction: 148459.03
Actual Value: 84900.0 	Prediction: 148459.03
Actual Value: 315500.0 	Prediction: 148459.03
Actual Value: 137500.0 	Prediction: 148459.03
Actual Value: 167240.0 	Prediction: 148459.03
Actual Value: 138000.0 	Prediction: 148459.03
Actual Value: 158000.0 	Prediction: 148459.03
Actual Value: 135500.0 	Prediction: 148459.03
Actual Value: 135000.0 	Prediction: 148459.03
Actual Value: 118500.0 	Prediction: 148459.03
Actual Value: 179200.0 	Prediction: 148459.03
Actual Value: 316600.0 	Prediction: 148459.03
Actual Value: 55993.0 	Prediction: 148459.03
Actual Value: 84000.0 	Prediction: 148459.03
Actual Value: 228000.0 	Prediction: 148459.03
Actual Value: 185000.0 	Prediction: 148459.03
Actual Value: 180000.0 	Prediction: 148459.03
Actual Value: 80000.0 	Prediction: 148459.03
Actual Value: 82500.0 	Prediction: 148459.03
Actual Value: 210000.0 	Prediction: 148

In [473]:
# Calculate RMSE and MAPE
rmse = np.sqrt(mean_squared_error(y_test_array.astype(np.int32), predictions.astype(np.int32)))
mape = mean_absolute_percentage_error(y_test_array.astype(np.int32), predictions.astype(np.int32))

print("RMSE:", rmse)
print("MAPE:", mape)


RMSE: 17268.481184453027
MAPE: 30.122913067420125


In [474]:
pred_df['RNN_Preds'] = np.round(predictions).astype(int)
pred_df.to_csv("preds.csv", index=False)

pred_df.head(10)

,Id,Actual_SalePrice,XGB_Preds,RNN_Preds
499,500,120000.0,218008,148459
1264,1265,181000.0,218114,148459
413,414,115000.0,217974,148459
1049,1050,84900.0,218062,148459
1036,1037,315500.0,219246,148459
615,616,137500.0,218234,148459
219,220,167240.0,218462,148459
1398,1399,138000.0,218033,148459
1194,1195,158000.0,218008,148459
887,888,135500.0,218008,148459


In [475]:
print(pred_df.Actual_SalePrice.min(), pred_df.XGB_Preds.min(), pred_df.RNN_Preds.min())
print(pred_df.Actual_SalePrice.mean(), pred_df.XGB_Preds.mean(), pred_df.RNN_Preds.mean())
print(pred_df.Actual_SalePrice.max(), pred_df.XGB_Preds.max(), pred_df.RNN_Preds.max())


40000.0 217828 148459
170352.24657534246 218237.6609589041 148459.0
625000.0 219735 148459


# RandomForestClassifier


In [498]:
# Define parameters for Random Forest
n_estimators = [700, 350]
max_depth = [8, 5, 3]
min_samples_split = [8, 5, 3]
min_samples_leaf = [10, 8, 5]

In [499]:
hyper_random = {"n_estimators":n_estimators,
                "max_depth":max_depth,
                "min_samples_split":min_samples_split,
                "min_samples_leaf":min_samples_leaf}

In [500]:
%%time
# Create gridsearch pipeline and fit model
clf_rf_tuned = GridSearchCV(RandomForestClassifier(), hyper_random,
                            cv = 5, verbose = 1,
                            n_jobs = -1)
clf_rf_tuned.fit(X_train, y_train)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


CPU times: user 9.56 s, sys: 1.2 s, total: 10.8 s
Wall time: 8min 14s


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [8, 5, 3], 'min_samples_leaf': [10, 8, 5],
                         'min_samples_split': [8, 5, 3],
                         'n_estimators': [700, 350]},
             verbose=1)

In [501]:
# Display best parameters of GridSearch
best_params_random = clf_rf_tuned.best_params_
print(best_params_random)

{'max_depth': 8, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 700}


In [502]:
# Set best parameters to the model
CV_clf_rf = RandomForestClassifier(max_depth=best_params_random["max_depth"],
                                   min_samples_leaf=best_params_random["min_samples_leaf"],
                                   min_samples_split=best_params_random["min_samples_split"],
                                   n_estimators= best_params_random["n_estimators"])

In [503]:
%%time
# Train model and make prediction
CV_clf_rf.fit(X_train, y_train)

y_test_predict_random = CV_clf_rf.predict(X_test)


CPU times: user 4.52 s, sys: 9.64 ms, total: 4.53 s
Wall time: 4.6 s


In [504]:
# Calculate RMSE and MAPE
rmse = np.sqrt(mean_squared_error(y_test_array.astype(np.int32), y_test_predict_random.astype(np.int32)))
mape = mean_absolute_percentage_error(y_test_array.astype(np.int32), y_test_predict_random.astype(np.int32))

print("RMSE:", rmse)
print("MAPE:", mape)

RMSE: 16579.206688330523
MAPE: 14.326554727257857


In [505]:
pred_df['RF_Preds'] = np.round(y_test_predict_random).astype(int)
pred_df.to_csv("preds.csv", index=False)

pred_df.head(10)

,Id,Actual_SalePrice,XGB_Preds,RNN_Preds,RF_Preds
499,500,120000.0,218008,148459,145000
1264,1265,181000.0,218114,148459,196000
413,414,115000.0,217974,148459,105000
1049,1050,84900.0,218062,148459,110000
1036,1037,315500.0,219246,148459,239000
615,616,137500.0,218234,148459,155000
219,220,167240.0,218462,148459,192000
1398,1399,138000.0,218033,148459,155000
1194,1195,158000.0,218008,148459,140000
887,888,135500.0,218008,148459,108000


In [507]:
print(pred_df.Actual_SalePrice.min(), pred_df.XGB_Preds.min(), pred_df.RNN_Preds.min(), pred_df.RF_Preds.min())
print(pred_df.Actual_SalePrice.mean(), pred_df.XGB_Preds.mean(), pred_df.RNN_Preds.mean(), pred_df.RF_Preds.mean())
print(pred_df.Actual_SalePrice.max(), pred_df.XGB_Preds.max(), pred_df.RNN_Preds.max(), pred_df.RF_Preds.max())


40000.0 217828 148459 55000
170352.24657534246 218237.6609589041 148459.0 164767.22260273973
625000.0 219735 148459 755000


In [508]:
pred_df[pred_df.Actual_SalePrice == 625000].head()

,Id,Actual_SalePrice,XGB_Preds,RNN_Preds,RF_Preds
1169,1170,625000.0,218321,148459,755000


In [509]:
pred_df[pred_df.Actual_SalePrice == 40000].head()

,Id,Actual_SalePrice,XGB_Preds,RNN_Preds,RF_Preds
30,31,40000.0,217828,148459,100000
